# Excerpt from the NGSolve i-tutorials (tutorial 2.7)
# 2.7 Facet spaces and hybrid methods

Mixed methods for second order problems lead to saddle point problems, and indefinite matrices. By hybridization one obtains a positive definite system again. It's structure is similar to the non-conforming $P^1$ method, but hybridization works for any order. See text-book by Brezzi and Fortin.

One skips the normal-continuity of the $H(div)$ variable, and reinforces it by a Lagrange parameter. This leads to the following discrete system:

Find $\sigma, u, \widehat u \in \Sigma_h \times V_h \times F_h$:

$$
\DeclareMathOperator{\Div}{div}
\begin{array}{ccccccll}
\sum_T \int \lambda^{-1} \sigma \tau & + & \sum_T \int_T \Div \tau \, u & + & \sum_F \int_F [\tau_n] \widehat u & = & 0 & \forall \, \tau \in \Sigma \\
\sum_T \int \Div \sigma \, v &&&&& = & \int f v & \forall \, v \in V_h \\
\sum_F \int [ \sigma_n ] \, \widehat v &&&&& = & \int_{\Gamma_n} g \widehat v & \forall \, \widehat v \in F_h
\end{array}
$$

where $\Sigma_h$ is an discontinuous $H(div)$ finite element space, $V_h$ a sub-space of $L_2$, and $F_h$ consists of polynomials on every edge.

In [ ]:
from netgen.geom2d import unit_square
from ngsolve import *
from draw import Draw
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))

same example as in 'mixed':

In [ ]:
source = sin(3.14*x)
ud = CoefficientFunction(5)
g = CoefficientFunction([0,0,0,y*(1-y)])
lam = 10

define spaces: 

* The *discontinuous* flag generates an element-wise $H(Div)$-space
* FacetFESpace lives only on facets (i.e. faces in 3D, edges in 2D, points in 1D)

Boundary conditions are now posed for the facet-space

In [ ]:
order = 4
V = HDiv(mesh, order=order, discontinuous=True)
# V = Discontinuous(HDiv(mesh, order=order))
Q = L2(mesh, order=order-1)
F = FacetFESpace(mesh, order=order, dirichlet="bottom")
X = FESpace([V,Q,F])
print ("sigmadofs:", X.Range(0))
print ("udofs:    ", X.Range(1))
print ("uhatdofs: ", X.Range(2))

Assemble forms. The jump-term is rewritten as
$$
\sum_F \int_F [\sigma_n] v = \sum_T \int_{\partial T} \sigma_n v
$$

In [ ]:
sigma,u,uhat = X.TrialFunction()
tau,v,vhat = X.TestFunction()

a = BilinearForm(X, condense=False)
a += (1/lam * sigma*tau + div(sigma)*v + div(tau)*u) * dx
n = specialcf.normal(mesh.dim)
a += (-sigma*n*vhat-tau*n*uhat) * dx(element_boundary=True)

f = LinearForm(X)
f += -source*v * dx - g*vhat.Trace() * ds

a.Assemble()
print ("A non-zero", a.mat.nze)

gf = GridFunction(X)

Solve system.

In [ ]:
f.Assemble()
gf.components[2].Set(ud, BND)

r = f.vec.CreateVector()
r.data = f.vec - a.mat * gf.vec
inv = a.mat.Inverse(freedofs=X.FreeDofs(a.condense))
if a.condense:
    r.data += a.harmonic_extension_trans * r
gf.vec.data += inv * r
if a.condense:
    gf.vec.data += a.harmonic_extension * gf.vec
    gf.vec.data += a.inner_solve * f.vec    

In [ ]:
gfsigma = gf.components[0]
Draw (0.1*gfsigma, mesh, "sigma",sd=2)

In [ ]:
gfu = gf.components[1]
Draw (gfu, mesh, "u",sd=2)

### Postprocessing:

#### primal solution (as reference):

In [ ]:
fesp = H1(mesh, order=order, dirichlet="bottom")
up, vp = fesp.TnT()

ap = BilinearForm(fesp)
ap += lam*grad(up)*grad(vp)*dx
ap.Assemble()

fp = LinearForm(fesp)
fp += source*vp*dx + g*vp * ds
fp.Assemble()

gfup = GridFunction(fesp, "u-primal")
gfup.Set(ud, BND)

r = fp.vec.CreateVector()
r.data = fp.vec - ap.mat * gfup.vec
gfup.vec.data += ap.mat.Inverse(freedofs=fesp.FreeDofs()) * r
Draw (gfup)

#### postprocessed solution:

In [ ]:
order_flux=order
fespost_u = L2(mesh, order=order_flux+1)
fespost_lam = L2(mesh, order=0)
fes_post = FESpace([fespost_u,fespost_lam])

u,la = fes_post.TrialFunction()
v,mu = fes_post.TestFunction()

a = BilinearForm(fes_post)
a += (lam*grad(u)*grad(v)+la*v+mu*u)*dx
a.Assemble()
f = LinearForm(fes_post)
f += (gfsigma*grad(v)+gfu*mu)*dx
f.Assemble()

gfpost = GridFunction(fes_post)
gfpost.vec.data = a.mat.Inverse() * f.vec

print ("err-upost:   ", sqrt(Integrate( (gfup-gfpost.components[0])**2, mesh)))

In [ ]:
Draw (gfpost.components[0], mesh, "upost")